In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import csv
import requests
import os
from zipfile import ZipFile
from datetime import date, datetime

In [2]:
yearDate = date.today().strftime('%m%d%y')
print(str(datetime.now()) + " Running 5_COVID_Hospital_ICU_Process")
print(yearDate)

2020-11-16 12:34:46.836806 Running 5_COVID_Hospital_ICU_Process
111620


In [3]:
dataFile_icu = "data/ICU/ICU_BedsCounty1_data{}.csv".format(yearDate)
dataFile_newhosp = "data/ICU/COVID_Hospitalizations_County1_data{}.csv".format(yearDate)
dataFile_hp = "data/ICU/Hospital_BedsCounty1_data{}.csv".format(yearDate)

In [4]:
df_ICU = pd.read_csv(dataFile_icu, sep=',',encoding='utf-8')
df_ICU.head()

,County,Measure Names,Measure Values
0,WASHINGTON,Total PediatricICU Capacity,0.0
1,WALTON,Total PediatricICU Capacity,0.0
2,VOLUSIA,Total PediatricICU Capacity,0.0
3,UNION,Total PediatricICU Capacity,0.0
4,TAYLOR,Total PediatricICU Capacity,0.0


In [5]:
df_newhosp = pd.read_csv(dataFile_newhosp,sep=',',encoding='utf-8')
df_newhosp.head()

,County,COVID Hospitalizations
0,WASHINGTON,5
1,WALTON,8
2,VOLUSIA,62
3,UNION,2
4,TAYLOR,0


In [6]:
df_hp = pd.read_csv(dataFile_hp, sep=',',encoding='utf-8')
df_hp.head()

,County,Measure Names,Measure Values
0,WASHINGTON,Total Staffed Bed Capacity,59.0
1,WALTON,Total Staffed Bed Capacity,115.0
2,VOLUSIA,Total Staffed Bed Capacity,1544.0
3,UNION,Total Staffed Bed Capacity,75.0
4,TAYLOR,Total Staffed Bed Capacity,12.0


In [7]:
df_ICU_pivot = pd.pivot_table(df_ICU,values='Measure Values',index= 'County',columns = 'Measure Names',aggfunc=np.sum)
df_ICU_pivot.head()

Measure Names,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity
County,,,,,,,,
ALACHUA,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0
BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
BAY,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0
BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
BREVARD,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0


In [8]:
df_hp_pivot = pd.pivot_table(df_hp,values='Measure Values',index= 'County',columns = 'Measure Names',aggfunc=np.sum)
df_hp_pivot.head()

Measure Names,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity
County,,,,
ALACHUA,287.0,0.164282,1460.0,1747.0
BAKER,114.0,0.182400,511.0,625.0
BAY,100.0,0.181818,450.0,550.0
BRADFORD,5.0,0.500000,5.0,10.0
BREVARD,506.0,0.312346,1114.0,1620.0


In [11]:
df_merge = pd.merge(df_ICU_pivot, df_hp_pivot, left_on=['County'],right_on=['County'], how ='outer')
df_merge2 = pd.merge(df_merge, df_newhosp, left_on=['County'],right_on= ['County'], how = 'outer')
df_merge2.head()

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations
0,ALACHUA,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0,287.0,0.164282,1460.0,1747.0,83
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,114.0,0.182400,511.0,625.0,0
2,BAY,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0,100.0,0.181818,450.0,550.0,24
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0
4,BREVARD,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0,506.0,0.312346,1114.0,1620.0,79


In [14]:
df_merge_sum = df_merge2.append(df_merge2.sum(numeric_only=True), ignore_index=True)

In [17]:
df_merge_sum['County'] = df_merge_sum['County'].replace(np.nan, 'A State')
df_merge_sum.iat[58,3] = df_merge_sum.iloc[58][2]/df_merge_sum.iloc[58][7]
df_merge_sum.iat[58,5] = df_merge_sum.iloc[58][4]/df_merge_sum.iloc[58][8]
df_merge_sum.iat[58,10] = df_merge_sum.iloc[58][9]/df_merge_sum.iloc[58][12]

In [18]:
df_merge_sum

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations
0,ALACHUA,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0,287.0,0.164282,1460.0,1747.0,83.0
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,114.0,0.182400,511.0,625.0,0.0
2,BAY,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0,100.0,0.181818,450.0,550.0,24.0
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0
4,BREVARD,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0,506.0,0.312346,1114.0,1620.0,79.0
5,BROWARD,354.0,107.0,0.232104,34.0,0.395349,52.0,461.0,86.0,1133.0,0.218010,4064.0,5197.0,284.0
6,CALHOUN,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2.0,0.100000,18.0,20.0,0.0
7,CHARLOTTE,50.0,5.0,0.090909,0.0,0.000000,0.0,55.0,0.0,198.0,0.316294,428.0,626.0,44.0
8,CITRUS,34.0,6.0,0.150000,0.0,0.000000,0.0,40.0,0.0,76.0,0.242038,238.0,314.0,47.0
9,CLAY,38.0,10.0,0.208333,2.0,0.500000,2.0,48.0,4.0,157.0,0.253226,463.0,620.0,29.0


In [19]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [20]:
df_merge_sum.to_excel(os.path.join(result_folder,"FL_County_Hospital_ICU{}.xlsx".format(yearDate)))

In [21]:
df_merge_sum['Avail_ICU']=(df_merge_sum['Available Adult ICU'] + df_merge_sum['Available Pediatric ICU'])
df_rename = df_merge_sum.rename(columns={"Adult ICU Census":"AICU_Census", "Available Adult ICU":"AICU_Avail",\
                                      "Available Adult ICU%":"A_ICU", "Pediatric ICU Census":"PICU_Census",\
                                      "Available Pediatric ICU":"PICU_Avail","Available Pediatric ICU%":"P_ICU",\
                                      "Total AdultICU Capacity":"T_AICU", "Total PediatricICU Capacity":"T_PICU",\
                                      "Available":"Bed_Avail", "Available Capacity":"Bed_AP",\
                                      "Total Staffed Bed Capacity":"T_bed", 'COVID Hospitalizations':'New_hosp'})
df_rename.head()

,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,Bed_Avail,Bed_AP,Bed Census,T_bed,New_hosp,Avail_ICU
0,ALACHUA,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0,287.0,0.164282,1460.0,1747.0,83.0,65.0
1,BAKER,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,114.0,0.182400,511.0,625.0,0.0,0.0
2,BAY,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0,100.0,0.181818,450.0,550.0,24.0,2.0
3,BRADFORD,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0
4,BREVARD,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0,506.0,0.312346,1114.0,1620.0,79.0,43.0


In [22]:
df_rename['COUNTYNAME'] = df_rename.loc[:, 'County']
df_rename['County']= df_rename['County'].str.title()

In [23]:
df_rename.loc[df_rename['County']=='A State','County']='State'
df_reset_index = df_rename.reset_index()
df_reset_index

,index,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,Bed_Avail,Bed_AP,Bed Census,T_bed,New_hosp,Avail_ICU,COUNTYNAME
0,0,Alachua,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0,287.0,0.164282,1460.0,1747.0,83.0,65.0,ALACHUA
1,1,Baker,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,114.0,0.182400,511.0,625.0,0.0,0.0,BAKER
2,2,Bay,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0,100.0,0.181818,450.0,550.0,24.0,2.0,BAY
3,3,Bradford,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0,BRADFORD
4,4,Brevard,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0,506.0,0.312346,1114.0,1620.0,79.0,43.0,BREVARD
5,5,Broward,354.0,107.0,0.232104,34.0,0.395349,52.0,461.0,86.0,1133.0,0.218010,4064.0,5197.0,284.0,141.0,BROWARD
6,6,Calhoun,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2.0,0.100000,18.0,20.0,0.0,0.0,CALHOUN
7,7,Charlotte,50.0,5.0,0.090909,0.0,0.000000,0.0,55.0,0.0,198.0,0.316294,428.0,626.0,44.0,5.0,CHARLOTTE
8,8,Citrus,34.0,6.0,0.150000,0.0,0.000000,0.0,40.0,0.0,76.0,0.242038,238.0,314.0,47.0,6.0,CITRUS
9,9,Clay,38.0,10.0,0.208333,2.0,0.500000,2.0,48.0,4.0,157.0,0.253226,463.0,620.0,29.0,12.0,CLAY


In [24]:
gdf_county_polygon = gpd.read_file("shp/Florida_Counties.shp")
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [25]:
a = set(df_reset_index['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'Miami-Dade', 'State'}

In [26]:
b.difference(a)

{'Dade',
 'Dixie',
 'Gilchrist',
 'Glades',
 'Hamilton',
 'Jefferson',
 'Lafayette',
 'Levy',
 'Liberty',
 'Wakulla'}

In [27]:
df_reset_index.loc[df_reset_index['County']=='Miami-Dade','County']='Dade'

In [28]:
a = set(df_reset_index['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'State'}

In [29]:
df_merge_poly = pd.merge(df_reset_index, gdf_county_polygon, left_on = ['County'],right_on = ['COUNTYNAME'], how = 'outer')
df_merge_poly.head()

,index,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,...,Avail_ICU,COUNTYNAME_x,OBJECTID,DEPCODE,COUNTY,COUNTYNAME_y,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,0.0,Alachua,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0,...,65.0,ALACHUA,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,1.0,Baker,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,BAKER,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,2.0,Bay,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0,...,2.0,BAY,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,3.0,Bradford,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,BRADFORD,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,4.0,Brevard,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0,...,43.0,BREVARD,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."


In [34]:
df_merge_poly1 = df_merge_poly.drop(columns=['OBJECTID','DEPCODE','COUNTY','DATESTAMP','COUNTYNAME_y','ShapeSTLen','ShapeSTAre'])
df_merge_poly2 = df_merge_poly1.rename(columns={"COUNTYNAME_x": "COUNTYNAME"})
df_merge_poly2.head(59)

,index,County,AICU_Census,AICU_Avail,A_ICU,PICU_Avail,P_ICU,PICU_Census,T_AICU,T_PICU,Bed_Avail,Bed_AP,Bed Census,T_bed,New_hosp,Avail_ICU,COUNTYNAME,geometry
0,0.0,Alachua,251.0,57.0,0.185065,8.0,0.170213,39.0,308.0,47.0,287.0,0.164282,1460.0,1747.0,83.0,65.0,ALACHUA,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,1.0,Baker,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,114.0,0.182400,511.0,625.0,0.0,0.0,BAKER,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,2.0,Bay,60.0,0.0,0.000000,2.0,0.500000,2.0,60.0,4.0,100.0,0.181818,450.0,550.0,24.0,2.0,BAY,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,3.0,Bradford,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5.0,0.500000,5.0,10.0,0.0,0.0,BRADFORD,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,4.0,Brevard,124.0,43.0,0.257485,0.0,0.000000,0.0,167.0,0.0,506.0,0.312346,1114.0,1620.0,79.0,43.0,BREVARD,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
5,5.0,Broward,354.0,107.0,0.232104,34.0,0.395349,52.0,461.0,86.0,1133.0,0.218010,4064.0,5197.0,284.0,141.0,BROWARD,"MULTIPOLYGON (((-80.22680 26.33478, -80.20374 ..."
6,6.0,Calhoun,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2.0,0.100000,18.0,20.0,0.0,0.0,CALHOUN,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."
7,7.0,Charlotte,50.0,5.0,0.090909,0.0,0.000000,0.0,55.0,0.0,198.0,0.316294,428.0,626.0,44.0,5.0,CHARLOTTE,"MULTIPOLYGON (((-81.97884 27.03517, -81.96591 ..."
8,8.0,Citrus,34.0,6.0,0.150000,0.0,0.000000,0.0,40.0,0.0,76.0,0.242038,238.0,314.0,47.0,6.0,CITRUS,"MULTIPOLYGON (((-82.47957 29.05252, -82.47889 ..."
9,9.0,Clay,38.0,10.0,0.208333,2.0,0.500000,2.0,48.0,4.0,157.0,0.253226,463.0,620.0,29.0,12.0,CLAY,"POLYGON ((-81.68008 30.12155, -81.67981 30.119..."


In [31]:
shpfile_name = "Florida_County_Hospital_ICU{}".format(yearDate)
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge = gpd.GeoDataFrame(df_merge_poly2, geometry='geometry',crs="EPSG:4326")
gdf_merge.to_file(os.path.join(result_folder,"{}.shp".format(shpfile_name)))

Saving Florida_County_Hospital_ICU111620.shp in results/


In [32]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))  

Saving Florida_County_Hospital_ICU111620.zip in publish/
Florida_County_Hospital_ICU111620.prj
Florida_County_Hospital_ICU111620.shx
Florida_County_Hospital_ICU111620.cpg
Florida_County_Hospital_ICU111620.shp
Florida_County_Hospital_ICU111620.dbf


In [33]:
print(str(datetime.now()) + " Done with 5_COVID_Hospital_ICU_Process")

2020-11-16 12:35:36.304837 Done with 5_COVID_Hospital_ICU_Process
